In [1]:
import numpy as np
import torch
import torch.nn as nn
from scipy.special import logsumexp
from typing import Tuple, Optional

In [3]:
class CurvedNeuralNetwork(nn.Module):
    """
    Curved Neural Network implementation with deformation parameter gamma
    Based on deformed exponential family distributions
    """
    
    def __init__(self, input_dim: int, hidden_dims: list, output_dim: int, 
                 gamma_prime: float = -0.5, beta: float = 1.0):
        super().__init__()
        
        self.gamma_prime = gamma_prime  # Deformation parameter
        self.beta = beta  # Inverse temperature
        self.N = input_dim  # System size for scaling
        
        # Build network layers
        layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.Tanh()  # Using tanh as in mean-field equations
            ])
            prev_dim = hidden_dim
        layers.append(nn.Linear(prev_dim, output_dim))
        
        self.network = nn.Sequential(*layers)
        
        # Initialize weights using Hebbian-like rule for memory patterns
        self._initialize_weights()
    
    def _initialize_weights(self):
        """Initialize weights with small random values"""
        for layer in self.network:
            if isinstance(layer, nn.Linear):
                nn.init.normal_(layer.weight, 0, 0.1)
                nn.init.zeros_(layer.bias)
    
    def effective_temperature(self, energy: torch.Tensor) -> torch.Tensor:
        """
        Compute effective temperature β'(x) = β / (1 - γβE(x))
        """
        gamma_scaled = self.gamma_prime / (self.N * self.beta)
        return self.beta / (1 - gamma_scaled * energy)
    
    def energy_function(self, x: torch.Tensor) -> torch.Tensor:
        """
        Compute energy E(x) for the curved network
        """
        # Simple quadratic energy for demonstration
        energy = -0.5 * torch.sum(x * x, dim=-1)
        return energy
    
    def deformed_activation(self, x: torch.Tensor, energy: torch.Tensor) -> torch.Tensor:
        """
        Apply deformed activation function with state-dependent temperature
        """
        beta_eff = self.effective_temperature(energy)
        
        # Deformed exponential: [1 + γx]^(1/γ)_+
        gamma_scaled = self.gamma_prime / (self.N * self.beta)
        
        if self.gamma_prime != 0:
            deformed_input = 1 + gamma_scaled * x
            # Ensure positivity
            deformed_input = torch.clamp(deformed_input, min=1e-8)
            output = torch.pow(deformed_input, 1/gamma_scaled)
        else:
            output = torch.exp(x)  # Standard exponential when γ → 0
            
        return output
    
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Forward pass with curved dynamics
        """
        energy = self.energy_function(x)
        beta_eff = self.effective_temperature(energy)
        
        # Standard forward pass
        output = self.network(x)
        
        # Apply deformed activation if in explosive regime
        if self.gamma_prime < 0:  # Explosive regime
            output = self.deformed_activation(output, energy)
        
        return output, beta_eff

In [4]:
class CurvedPACBayes:
    """
    PAC-Bayes framework for curved neural networks
    Derives generalization bounds accounting for curvature effects
    """
    
    def __init__(self, network: CurvedNeuralNetwork, m: int, delta: float = 0.05):
        self.network = network
        self.m = m  # Training set size
        self.delta = delta  # Confidence parameter
        
    def kl_divergence_curved(self, posterior_params: dict, prior_params: dict) -> float:
        """
        Compute KL divergence between curved distributions
        KL(ρ||π) where ρ is posterior, π is prior
        """
        # For Gaussian posterior and prior (simplified)
        mu_post, sigma_post = posterior_params['mu'], posterior_params['sigma']
        mu_prior, sigma_prior = prior_params['mu'], prior_params['sigma']
        
        # Standard KL for Gaussians, modified by curvature
        kl_standard = 0.5 * (
            torch.log(sigma_prior / sigma_post) + 
            (sigma_post**2 + (mu_post - mu_prior)**2) / sigma_prior**2 - 1
        ).sum()
        
        # Curvature correction factor
        gamma_correction = 1 + abs(self.network.gamma_prime) * kl_standard / self.network.N
        
        return kl_standard * gamma_correction
    
    def empirical_risk_curved(self, data_loader, loss_fn) -> float:
        """
        Compute empirical risk accounting for curved dynamics
        """
        total_loss = 0.0
        total_samples = 0
        
        self.network.eval()
        with torch.no_grad():
            for x, y in data_loader:
                output, beta_eff = self.network(x)
                loss = loss_fn(output, y)
                
                # Weight loss by effective temperature (explosive dynamics)
                if self.network.gamma_prime < 0:
                    loss = loss * beta_eff.mean()  # Accelerated convergence
                
                total_loss += loss.item() * x.size(0)
                total_samples += x.size(0)
        
        return total_loss / total_samples
    
    def pac_bayes_bound_mcallester(self, empirical_risk: float, kl_div: float) -> float:
        """
        McAllester PAC-Bayes bound modified for curved networks
        """
        # Standard McAllester bound
        complexity_term = (kl_div + np.log(2 * np.sqrt(self.m) / self.delta)) / (2 * self.m - 1)
        
        # Curvature modification for explosive regime
        if self.network.gamma_prime < 0:
            # Explosive networks have enhanced generalization in certain regimes
            explosive_factor = 1 - abs(self.network.gamma_prime) * 0.1  # Conservative estimate
            complexity_term *= explosive_factor
        
        return empirical_risk + np.sqrt(complexity_term)
    
    def pac_bayes_bound_catoni(self, empirical_risk: float, kl_div: float, 
                               lambda_param: float = 1.0) -> float:
        """
        Catoni PAC-Bayes bound for curved networks
        """
        # Catoni's bound with λ parameter
        kl_term = (kl_div + np.log(2 * np.sqrt(self.m) / self.delta)) / (lambda_param * self.m)
        
        # For curved networks, adjust λ based on effective temperature dynamics
        if hasattr(self.network, 'last_beta_eff'):
            avg_beta_eff = self.network.last_beta_eff.mean().item()
            lambda_adjusted = lambda_param * avg_beta_eff
        else:
            lambda_adjusted = lambda_param
            
        # Catoni's exponential bound
        if kl_term < 1:
            bound = empirical_risk + kl_term + kl_term**2 / (2 * (1 - kl_term))
        else:
            bound = 1.0  # Vacuous bound
            
        return bound
    
    def stability_based_bound(self, data_loader, perturbation_scale: float = 0.01) -> float:
        """
        Stability-based generalization bound for explosive networks
        Leverages the self-regulating annealing property
        """
        original_params = {}
        for name, param in self.network.named_parameters():
            original_params[name] = param.clone()
        
        # Compute original loss
        original_loss = self.empirical_risk_curved(data_loader, nn.MSELoss())
        
        # Perturb parameters and compute loss
        perturbed_losses = []
        for _ in range(10):  # Multiple perturbations
            for name, param in self.network.named_parameters():
                noise = torch.randn_like(param) * perturbation_scale
                param.data += noise
            
            perturbed_loss = self.empirical_risk_curved(data_loader, nn.MSELoss())
            perturbed_losses.append(perturbed_loss)
            
            # Restore original parameters
            for name, param in self.network.named_parameters():
                param.data = original_params[name]
        
        # Stability measure
        stability = np.std(perturbed_losses)
        
        # Bound based on stability (simplified)
        stability_bound = original_loss + 2 * stability * np.sqrt(np.log(1/self.delta) / (2 * self.m))
        
        return stability_bound


In [5]:
class CurvedNetworkTrainer:
    """
    Training framework that optimizes both performance and PAC-Bayes bounds
    """
    
    def __init__(self, network: CurvedNeuralNetwork, pac_bayes: CurvedPACBayes):
        self.network = network
        self.pac_bayes = pac_bayes
        self.training_losses = []
        self.generalization_bounds = []
    
    def pac_bayes_objective(self, data_loader, loss_fn, lambda_reg: float = 0.1):
        """
        Combined objective: empirical risk + PAC-Bayes regularization
        """
        # Empirical risk
        emp_risk = self.pac_bayes.empirical_risk_curved(data_loader, loss_fn)
        
        # Current parameters as posterior
        posterior_params = {
            'mu': torch.cat([p.flatten() for p in self.network.parameters()]),
            'sigma': torch.ones_like(torch.cat([p.flatten() for p in self.network.parameters()])) * 0.1
        }
        
        # Prior (zero mean, unit variance)
        prior_params = {
            'mu': torch.zeros_like(posterior_params['mu']),
            'sigma': torch.ones_like(posterior_params['mu'])
        }
        
        # KL divergence
        kl_div = self.pac_bayes.kl_divergence_curved(posterior_params, prior_params)
        
        # PAC-Bayes bound
        bound = self.pac_bayes.pac_bayes_bound_mcallester(emp_risk, kl_div.item())
        
        # Combined objective
        objective = emp_risk + lambda_reg * bound
        
        return objective, emp_risk, bound
    
    def train_epoch(self, data_loader, optimizer, lambda_reg: float = 0.1):
        """
        Train for one epoch with PAC-Bayes regularization
        """
        self.network.train()
        epoch_loss = 0.0
        
        for batch_idx, (x, y) in enumerate(data_loader):
            optimizer.zero_grad()
            
            # Forward pass
            output, beta_eff = self.network(x)
            
            # Store effective temperature for bound computation
            self.network.last_beta_eff = beta_eff
            
            # PAC-Bayes objective
            objective, emp_risk, bound = self.pac_bayes_objective(
                data_loader, nn.MSELoss(), lambda_reg
            )
            
            # Backward pass
            if isinstance(objective, torch.Tensor):
                objective.backward()
            else:
                # Convert to tensor if needed
                loss_tensor = torch.tensor(objective, requires_grad=True)
                loss_tensor.backward()
            
            optimizer.step()
            
            epoch_loss += objective if isinstance(objective, float) else objective.item()
            
            # Track explosive dynamics
            if self.network.gamma_prime < 0 and batch_idx % 100 == 0:
                print(f"Batch {batch_idx}: β_eff = {beta_eff.mean().item():.4f}, "
                      f"Bound = {bound:.4f}")
        
        return epoch_loss / len(data_loader)
    
    def evaluate_generalization(self, train_loader, test_loader):
        """
        Evaluate generalization performance and bounds
        """
        # Training performance
        train_risk = self.pac_bayes.empirical_risk_curved(train_loader, nn.MSELoss())
        
        # Test performance
        test_risk = self.pac_bayes.empirical_risk_curved(test_loader, nn.MSELoss())
        
        # PAC-Bayes bounds
        posterior_params = {
            'mu': torch.cat([p.flatten() for p in self.network.parameters()]),
            'sigma': torch.ones_like(torch.cat([p.flatten() for p in self.network.parameters()])) * 0.1
        }
        prior_params = {
            'mu': torch.zeros_like(posterior_params['mu']),
            'sigma': torch.ones_like(posterior_params['mu'])
        }
        
        kl_div = self.pac_bayes.kl_divergence_curved(posterior_params, prior_params)
        
        mcallester_bound = self.pac_bayes.pac_bayes_bound_mcallester(train_risk, kl_div.item())
        catoni_bound = self.pac_bayes.pac_bayes_bound_catoni(train_risk, kl_div.item())
        stability_bound = self.pac_bayes.stability_based_bound(train_loader)
        
        results = {
            'train_risk': train_risk,
            'test_risk': test_risk,
            'generalization_gap': test_risk - train_risk,
            'mcallester_bound': mcallester_bound,
            'catoni_bound': catoni_bound,
            'stability_bound': stability_bound,
            'kl_divergence': kl_div.item(),
            'gamma_prime': self.network.gamma_prime
        }
        
        return results


In [6]:
def run_curved_network_experiment():
    """
    Demonstrate curved neural networks with explosive dynamics
    """
    # Generate synthetic data
    np.random.seed(42)
    torch.manual_seed(42)
    
    n_samples = 1000
    input_dim = 20
    
    X = torch.randn(n_samples, input_dim)
    # Non-linear target with memory-like patterns
    y = torch.sin(X.sum(dim=1, keepdim=True)) + 0.1 * torch.randn(n_samples, 1)
    
    # Split data
    train_size = int(0.8 * n_samples)
    train_X, test_X = X[:train_size], X[train_size:]
    train_y, test_y = y[:train_size], y[train_size:]
    
    train_dataset = torch.utils.data.TensorDataset(train_X, train_y)
    test_dataset = torch.utils.data.TensorDataset(test_X, test_y)
    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    # Test different gamma values
    gamma_values = [0.0, -0.5, -1.0, -1.5]  # Explosive regime for negative values
    results = {}
    
    for gamma in gamma_values:
        print(f"\n=== Testing γ' = {gamma} ===")
        
        # Create network
        network = CurvedNeuralNetwork(
            input_dim=input_dim,
            hidden_dims=[64, 32],
            output_dim=1,
            gamma_prime=gamma,
            beta=1.0
        )
        
        # PAC-Bayes framework
        pac_bayes = CurvedPACBayes(network, m=train_size)
        trainer = CurvedNetworkTrainer(network, pac_bayes)
        
        # Training
        optimizer = torch.optim.Adam(network.parameters(), lr=0.001)
        
        for epoch in range(50):
            loss = trainer.train_epoch(train_loader, optimizer, lambda_reg=0.1)
            if epoch % 10 == 0:
                print(f"Epoch {epoch}: Loss = {loss:.6f}")
        
        # Evaluation
        eval_results = trainer.evaluate_generalization(train_loader, test_loader)
        results[gamma] = eval_results
        
        print(f"Train Risk: {eval_results['train_risk']:.6f}")
        print(f"Test Risk: {eval_results['test_risk']:.6f}")
        print(f"Generalization Gap: {eval_results['generalization_gap']:.6f}")
        print(f"McAllester Bound: {eval_results['mcallester_bound']:.6f}")
        print(f"Catoni Bound: {eval_results['catoni_bound']:.6f}")
        print(f"Stability Bound: {eval_results['stability_bound']:.6f}")
    
    return results

# Run experiment
if __name__ == "__main__":
    results = run_curved_network_experiment()
    
    # Analyze explosive behavior
    print("\n=== Analysis of Explosive Behavior ===")
    for gamma, result in results.items():
        bound_tightness = result['mcallester_bound'] - result['generalization_gap']
        print(f"γ' = {gamma:4.1f}: Bound Tightness = {bound_tightness:.6f}, "
              f"KL Div = {result['kl_divergence']:.6f}")



=== Testing γ' = 0.0 ===
Epoch 0: Loss = 0.733523
Epoch 10: Loss = 0.733523
Epoch 20: Loss = 0.733523
Epoch 30: Loss = 0.733523
Epoch 40: Loss = 0.733523
Train Risk: 0.557970
Test Risk: 0.500709
Generalization Gap: -0.057261
McAllester Bound: 1.755530
Catoni Bound: 1.000000
Stability Bound: 0.558197

=== Testing γ' = -0.5 ===
Batch 0: β_eff = 1.3416, Bound = 11.0445
Epoch 0: Loss = 3.260886
Batch 0: β_eff = 1.3567, Bound = 11.0432
Batch 0: β_eff = 1.3441, Bound = 11.0443
Batch 0: β_eff = 1.3578, Bound = 11.0425
Batch 0: β_eff = 1.3918, Bound = 11.0452
Batch 0: β_eff = 1.3526, Bound = 11.0436
Batch 0: β_eff = 1.3621, Bound = 11.0452
Batch 0: β_eff = 1.3501, Bound = 11.0422
Batch 0: β_eff = 1.3688, Bound = 11.0442
Batch 0: β_eff = 1.3192, Bound = 11.0414
Batch 0: β_eff = 1.3767, Bound = 11.0426
Epoch 10: Loss = 3.260114
Batch 0: β_eff = 1.3501, Bound = 11.0439
Batch 0: β_eff = 1.3673, Bound = 11.0428
Batch 0: β_eff = 1.3501, Bound = 11.0423
Batch 0: β_eff = 1.3908, Bound = 11.0420
Batch